In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import kraft

In [ ]:
fu = kraft.information.get_ic

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
def simulate(sh, n_ca=None):

    if n_ca is None:

        vector = np.random.normal(size=sh[1])

        matrix = np.random.normal(size=sh)

    else:

        vector = np.random.randint(0, high=n_ca, size=sh[1])

        matrix = np.random.randint(0, high=n_ca, size=sh)

    la1_ = pd.Index(("Label {}".format(i) for i in range(sh[1])), name="Axis 1")

    return (
        pd.Series(vector, index=la1_, name="Vector"),
        pd.DataFrame(
            matrix,
            pd.Index(("Label {}".format(i) for i in range(sh[0])), name="Axis 0"),
            columns=la1_,
        ),
    )

## make

In [ ]:
(ta, da) = simulate([32, 16])

ta.iloc[-8::2] = np.nan

kraft.function_heat_map.make(ta, da, fu, ac=None)

In [ ]:
(ta, da) = simulate([16, 32])

da.iloc[:, 0] = np.nan

da.iloc[-8:, -8:] = np.nan

kraft.function_heat_map.make(ta, da, fu)

In [ ]:
da = simulate([16, 8])[1]

for nu_ in [[1, 2, 2, 2, 2, 0, 0, 0], [1, 1, 2, 2, 2, 0, 0, 0]]:

    ta = pd.Series(nu_, index=da.columns)

    for ac in [None, True, False]:

        ta.name = "ac={}".format(ac)

        kraft.function_heat_map.make(ta, da, fu, ac=ac, tyta="categorical")

In [ ]:
ta = pd.read_csv("data/function_heat_map_ta.tsv", "\t", index_col=0, squeeze=True)

da = pd.read_csv("data/function_heat_map_da.tsv", "\t", index_col=0)

In [ ]:
for n_row in [1, 2, 3]:

    kraft.function_heat_map.make(
        da.iloc[0, :], da.iloc[:n_row, :], fu, n_sa=0, n_sh=0, st=3
    )

In [ ]:
kraft.function_heat_map.make(ta, da, fu, n_jo=8, n_sa=10, n_sh=10, tyta="binary")

In [ ]:
for shape in ([1, 8], [2, 8], [8, 8], [64, 8], [8, 2], [8, 4], [8, 256]):

    (ta, da) = simulate(shape)

    kraft.function_heat_map.make(ta, da, fu, n_pl=None)

## summarize

In [ ]:
n_co = 16

(ta, da) = simulate([16, n_co])

da0n = simulate([8, n_co], 4)[1]

da01 = simulate([4, n_co], 2)[1]

da.drop(da.columns.values[-3], axis=1, inplace=True)

da0n.drop(da0n.columns.values[-2], axis=1, inplace=True)

da01.drop(da01.columns.values[-1], axis=1, inplace=True)

pa_da = [
    {
        "name": "Continuous",
        "data": da,
        "type": "continuous",
    },
    {
        "name": "Categorical",
        "data": da0n,
        "type": "categorical",
    },
    {
        "name": "Binary",
        "data": da01,
        "type": "binary",
    },
]

for ve in pa_da:

    ve["statistic"] = kraft.function_heat_map.make(ta, ve["data"], fu, pl=False)

for it in (False, True):

    kraft.function_heat_map.summarize(ta, pa_da, it=it)